In [1]:
import os

import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

# Project overview
Energy is used on a daily basis for phones, computers, washing machines, heaters and a vast array of appliances.
Our dependence on electricity makes it critical to accurately predict how much will need to generate on any given day.
Hence, our project aims to correlate and model temperature's effect on energy demand.
We will begin by importing and cleaning our two datasets, before analysing the data integrity and creating visualisations to intuitively highlight the impact several variables have on Australia's net energy useage.
These visualisations aim to show not only energy and temperature, but yearly trends, seasonal shifts and other potential anomalies.
After this we create a basic timeseries model, to evaluate precisely how well we're able to measure energy demand on a day-to-day basis.
This report will conclude with a summary of our results (including the significance of different independent and extraneous variables) and a series of suggestions on how our work can be improved going forwards.

To help achieve these results we will use a range of technical and statistical tools.
We heavily rely on Python and its associated libraries (largely Pandas for data cleaning and management, Numpy for numerical calculations, MatPlotLib for graphs and Scikit learn for machine learning modelling).

# Supplied data
Our weather data is collected from the Bureau of Meteorology's (BOM) Automatic Weather Stations and energy data from The Australian Energy Market Operator (AEMO).
The data comes in the form of a series of CSVs containing measurements every 30 minutes.
The weather data contains precipitation (mm), temperature (°C), relative humidity (%), wind speed (km/h), wind direction (° true), maximum windgust speed (km/h), pressure (hPa) and whether manual/automatic measurements were taken.
It is quite common for there to be multiple features (columns) with nearly identical data (i.e. different forms of temperature or pressure).
Energy data provides total demand and a RRP (energy price).

The large portion of this data will be processed to elliminate any present trends, biases or inconsistancies. Yet, not all the provided data from BOM and AEMO are relavent (so many columns are removed).

# Data Cleaning
## Import the data
When importing data, we first need to find a list of all the data files/their paths.
We can then merge the separate spreadsheets into two dataframes (one for temperature and one for energy).
The process of finding the file paths, loading the seperate CSV files and then finally concatenating them together can be tedious, however this ultimately allows for easy access to our data.

In [2]:
energy_locations = os.listdir("../Data/Energy")
temperature_locations = os.listdir("../Data/Temperature")

energy_CSVs = [pd.read_csv("../Data/Energy/" + location) for location in energy_locations]
temperature_CSVs = [pd.read_csv("../Data/Temperature/" + location) for location in temperature_locations if "Data" in location]

/home/kamron/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,24,26,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/kamron/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,22,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/kamron/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (12,14,18,20,28,32) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/kamron/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (12,16,22,24,26,28,30) have mixed types.Specify dtype option on import or set low_memory=False.
  i

In [3]:
energy_data = pd.concat(energy_CSVs, ignore_index=True)
temperature_data = pd.concat(temperature_CSVs, ignore_index=True)

# Note some excess data is shaved off here for simplicity
stations_data = pd.read_csv("../Data/Temperature/HM01X_StnDet_999999999743964.txt", skiprows=5, header=None, usecols=range(1, 21), names=["StationNumber", "RainfulDistrict", "Station", "StationOpenDate", "StationCloseDate", "Latitude", "Longitude", "LocationMethod", "State", "StationHeight", "BarometerHeight", "WMO", "FirstYear", "LastYear", "CompletionPercent", "YQualityPercent", "NQualityPercent", "WQualityPercent", "SQualityPercent", "IQualityPercent"])

## Column standardising
To ensure it is easy to find specific data features, all column names are standardised.
We have chosen to use PascalCase here.

As there are two sets of dates available here, we append a 1 to the end of the columns.

In [26]:
energy_data.columns
temperature_data.columns
stations_data.columns

Index(['Region', 'Date', 'TotalDemand', 'RRP'], dtype='object')

Index(['Precipitation', 'AirTemperature', 'WetBulbTemperature',
       'DewTemperature', 'RelativeHumidity', 'SeaPressure', 'StationPressure',
       'AWSFlag', 'Date', 'Region'],
      dtype='object')

Index(['StationNumber', 'RainfulDistrict', 'Station', 'Latitude', 'Longitude',
       'LocationMethod', 'State', 'StationHeight', 'BarometerHeight', 'WMO',
       'CompletionPercent'],
      dtype='object')

In [5]:
energy_data.columns = ["Region", "Date", "TotalDemand", "RRP", "PeriodType"]
temperature_data.columns = [
    "HM", "StationNumber", "Year1", "Month1", "Day1", "Hour1", "Minute1", "Year", "Month", "Day", "Hour", "Minute", "Precipitation", "PrecipitationQuality",
    "AirTemperature", "AirTemperatureQuality", "WetBulbTemperature", "WetBulbTemperatureQuality", "DewTemperature", "DewTemperatureQuality", "RelativeHumidity",
    "RelativeHumidityQuality", "WindSpeed", "WindSpeedQuality", "WindDirection", "WindDirectionQuality", "WindgustSpeed", "WindgustSpeedQuality", "SeaPressure",
    "SeaPressureQuality", "StationPressure", "StationPressureQuality", "AWSFlag", "#"
]

## Remove any easily found unnecessary data
Our dataset contains a multitude of columns, and several have absolutely no information within them.
As these feature columns often contain a variety of null types (sometimes Numpy's NaN, sometimes an empty string and sometimes a 0 for a non-integer or category column), we analyse the number of unique elements.

Reducing the number of columns present additionally makes it easy to quickly look at the state of the data when moving forward.

In [ ]:
def remove_non_uniques(dataframe: pd.DataFrame, filter = []):
    remove = [name for name, series in dataframe.items() if len(series.unique()) <= 2 and not name in filter]
    dataframe.drop(remove, axis=1, inplace=True)
    return remove

print("Removed:")
remove_non_uniques(energy_data)
remove_non_uniques(temperature_data)
remove_non_uniques(stations_data, "LocationMethod")

# Manually remove extra columns
stations_data.drop("StationOpenDate", axis=1, inplace=True)

In [8]:
energy_data.columns
temperature_data.columns
stations_data.columns

Index(['Region', 'Date', 'TotalDemand', 'RRP'], dtype='object')

Index(['StationNumber', 'Year1', 'Month1', 'Day1', 'Hour1', 'Minute1', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Precipitation', 'AirTemperature',
       'WetBulbTemperature', 'DewTemperature', 'RelativeHumidity', 'WindSpeed',
       'WindDirection', 'WindgustSpeed', 'SeaPressure', 'StationPressure',
       'AWSFlag'],
      dtype='object')

Index(['StationNumber', 'RainfulDistrict', 'Station', 'Latitude', 'Longitude',
       'LocationMethod', 'State', 'StationHeight', 'BarometerHeight', 'WMO',
       'CompletionPercent'],
      dtype='object')

## Deal with multiple dates
The provided data contains two sets of date values.
The provided notes state that first dates are local and the second are local standard.
Whilst generally identical, local time incorporates day light saving (shifting the clock back/foward by an hour).
For simplicity, local standard time is used.

In [9]:
# Remove extra dates
temperature_data.drop(["Year1", "Month1", "Day1", "Hour1", "Minute1"], axis=1, inplace=True)

# Reformat dates into Pandas' datatime64 objects
# Replacing old format
temperature_data["Date"] = pd.to_datetime(temperature_data[["Year", "Month", "Day", "Hour", "Minute"]])
energy_data["Date"] = pd.to_datetime(energy_data["Date"])

temperature_data.drop(["Year", "Month", "Day", "Hour", "Minute"], axis=1, inplace=True)

## Check for duplicated rows
On top of finding and removing useless columns, we can also drop useless rows.
This is far more straightforward as the provided Pandas functions work flawlessly with almost no manual input.

In [10]:
energy_data.drop_duplicates(inplace=True)
temperature_data.drop_duplicates(inplace=True)
stations_data.drop_duplicates(inplace=True)

## Converting datatypes
The next step is to convert the data types. 
To ensure that we can plot and model our data, we use specific Pandas/Python datatypes.
The use of specific datatypes like DateTime and Category are crucial in providing a simple but flexible API.
This will latter allow indexing through dates (so ```loc``` with multiple conditional statements aren't  repeatidly used)!

Although Pandas includes functions for parsing data, these are of little use because of the special scenareos present here.
These include spaces before and after numbers and hashtags in a random location.
To easily overcome these issues a function is created which finds all columns with generic objects, and performs some operation on each entry.
This works specifically because Pandas does not set a specific datatype for strings.

For AWS Flag, we assume that null values are recorded manually, as this is the worst case scenareo.

In [11]:
def to_object_columns(lambda_function):
    string_columns = temperature_data.select_dtypes("object").columns
    temperature_data[string_columns] = temperature_data[string_columns].apply(lambda_function)

In [12]:
to_object_columns(lambda column: column.str.strip())

In [13]:
temperature_data["AWSFlag"] = temperature_data["AWSFlag"].replace("", 0).astype("category")
temperature_data["AWSFlag"].fillna(0, inplace=True)
temperature_data["RelativeHumidity"] = temperature_data["RelativeHumidity"].replace("###", np.NaN)

In [14]:
to_object_columns(lambda column: pd.to_numeric(column))

In [15]:
temperature_data.dtypes

StationNumber                  int64
Precipitation                float64
AirTemperature               float64
WetBulbTemperature           float64
DewTemperature               float64
RelativeHumidity             float64
WindSpeed                    float64
WindDirection                float64
WindgustSpeed                float64
SeaPressure                  float64
StationPressure              float64
AWSFlag                     category
Date                  datetime64[ns]
dtype: object

## Finding missing data
Through analysing where and how our data is missing, we are able conclude what is worth keeping and what should be removed/interpolated.
We of course can only do this for the temperature data, as there is no missing energy data.

As we can tell, a large chunk of wind data is missing and so these columns must be removed.

If we graph out each time series (in the graphing notebook), we notice that the missing data for the other columns (like wet bulb temperature) is decently randomly distributed.
This means that it should be relatively safe to interpolate for the missing values (i.e. reason using linear algebra what the value should be).

In [16]:
def get_null_counts(dataframe: pd.DataFrame):
    return dataframe.isnull().mean()[dataframe.isnull().mean() > 0]

In [17]:
get_null_counts(energy_data)
get_null_counts(temperature_data)

Series([], dtype: float64)

Precipitation         0.229916
AirTemperature        0.444437
WetBulbTemperature    0.011324
DewTemperature        0.375311
RelativeHumidity      0.375312
WindSpeed             0.532966
WindDirection         0.432305
WindgustSpeed         0.403183
SeaPressure           0.137730
StationPressure       0.011135
dtype: float64

In [18]:
pd.to_pickle([energy_data, temperature_data, stations_data], "../Data/CleanedData.pickle")

In [19]:
temperature_data.drop(["WindSpeed", "WindgustSpeed", "WindDirection"], axis=1, inplace=True)

# Note that using inplace currently throws an error
# So interpolated columns must be manually overridden
missing_columns = list(get_null_counts(temperature_data).keys())
temperature_data[missing_columns] = temperature_data[missing_columns].interpolate(method="linear")

## Grouping data
Energy and temperature data are recorded in specific regions, however they use different keys/criteria (so we must align them ourselves).
This is the intermediary stage before the two datasets can be joined together.

More than one weather station existed in Victoria (for a short time) and here both stations data are just added together.
*Although this may lead to a slightly biased dataset (if there ends up being slightly more data for Victoria than other states), using seperate models for seperate states will eliminate this issue*.

In [20]:
energy_data["Region"].unique()
temperature_data["StationNumber"].unique()

array(['VIC1', 'SA1', 'TAS1', 'QLD1', 'NSW1'], dtype=object)

array([94029, 86071, 66062, 40913, 86338, 23090])

In [21]:
region_remove_number_map = {"SA1": "SA", "QLD1": "QLD", "NSW1": "NSW", "VIC1": "VIC", "TAS1": "TAS"}
station_to_region_map = {23090: "SA", 40913: "QLD", 66062: "NSW", 86071: "VIC", 94029: "TAS", 86338: "VIC"}

temperature_data["Region"] = temperature_data["StationNumber"].map(station_to_region_map)
energy_data["Region"] = energy_data["Region"].map(region_remove_number_map)

temperature_data.drop("StationNumber", axis=1, inplace=True)

## Combining energy and temperature data
To be able to create a model which predicts energy demand using temperature data we need to align our temperature and energy measurements together (i.e. by date).
This is accomplished through closest value merges.
These merges work with sorted data, so we begin by sorting our data by date.
We can then use the ```merge_asof``` function (where we specify that our data is grouped ```by``` region) to merge together the two datasets (where the closest entries get combined).
As our data is nearly always recorded in 30 minute intervals, we only merge two rows if there individual dates differ by less than 30 minutes.

Any unpaired rows result in null values, and since we have a small number of them we can tell that our merge happened successfully!

In [22]:
energy_data.sort_values(by="Date", inplace=True)
temperature_data.sort_values(by="Date", inplace=True)

data = pd.merge_asof(energy_data, temperature_data, on="Date", by="Region", tolerance=pd.Timedelta("30 min"))
data.set_index("Date", inplace=True)

In [23]:
get_null_counts(data)
data.dropna(inplace=True)

Precipitation         0.001634
AirTemperature        0.001634
WetBulbTemperature    0.001634
DewTemperature        0.001634
RelativeHumidity      0.001634
SeaPressure           0.001634
StationPressure       0.001634
AWSFlag               0.001634
dtype: float64

In [24]:
data

,Region,TotalDemand,RRP,Precipitation,AirTemperature,WetBulbTemperature,DewTemperature,RelativeHumidity,SeaPressure,StationPressure,AWSFlag
Date,,,,,,,,,,,
2000-01-01 00:30:00,NSW,6763.57000,15.64,0.000000,14.413604,14.1,10.539772,77.829696,1011.100000,1006.4,0
2000-01-01 00:30:00,SA,1328.68667,38.54,1.239962,13.200000,10.1,6.800000,65.000000,1018.700000,1012.6,0
2000-01-01 00:30:00,QLD,3905.56833,39.40,0.000000,20.686886,19.2,17.185578,80.606576,1007.200000,1006.2,0
2000-01-01 00:30:00,VIC,4419.03667,9.97,0.000000,13.700000,10.5,7.200000,65.000000,1017.000000,1013.1,0
2000-01-01 01:00:00,SA,1375.14833,38.54,1.239886,13.400000,10.1,6.500000,63.000000,1018.500000,1012.4,0
...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01 00:00:00,QLD,6218.39000,52.15,0.000000,18.530150,21.6,11.839619,65.000000,1019.643632,1013.8,0
2020-01-01 00:00:00,VIC,4372.27000,85.23,1.313453,13.143757,12.3,6.642222,65.000000,1014.900000,1014.0,0
2020-01-01 00:00:00,SA,1474.11000,85.95,0.000000,17.000000,12.3,8.700000,61.000000,1019.900000,1010.6,0


## Saving final model

In [25]:
pd.to_pickle(data, "../Data/Data.pickle")